# Welding Defect Classification

In this notebook, we build a deep learning model to classify images of welds as defective or not defective. We make use of a pre-trained model with transfer learning.

### Load Libraries

Load necessary libraries, and optionally install missing requirements.  If any other needed libraries are not installed, please install them with `!pip install <library name>`.

We also print out some data about the environment and set the seed.

In [ ]:
# # Install the following libraries with pip, if not already available:
!pip install matplotlib
!pip install tf2onnx

import os

import numpy as np
from numpy.random import seed
import tensorflow as tf

from tensorflow.keras import backend as K
from tensorflow.keras import applications, layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef, accuracy_score
from PIL import Image

import tf2onnx
import onnx

from time import perf_counter

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Print library versions
print('numpy:', np.__version__)
print('tensorflow:', tf.__version__)

# Print backend and image data format
print('backend:', K.backend())
print('image data format:', K.image_data_format())

# Set seed 
seed(1)
tf.compat.v1.set_random_seed(2)

### Define functions and set environment and model training variables

In [ ]:
def plot_history(model_hist):
    '''Given model training data, plot the model training history.'''
    plt.plot(model_hist.history['accuracy'])
    plt.plot(model_hist.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    return

def print_performance(y_truth, y_pred_class):
    '''Given ground truth and predicted labels, print the performance of the model:
        - Confusion Matrix
        - Accuracy
        - Matthews Correlation Coefficient
    '''
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_truth, y_pred_class))

    print("Accuracy: {0:.4f}".format(accuracy_score(y_truth, y_pred_class)))
    print("MCC:      {0:.4f}".format(matthews_corrcoef(y_truth, y_pred_class)))
    return


def list_files(path):
    '''List files recursively for a given path. Use full path for each file.'''
    files = []
    for r, d, f in os.walk(path):
        for file in f:
            files.append(os.path.join(r, file))
    return files

def print_pil_img_info(an_img_path):
    '''Print info for image opened with PIL Image'''
    an_img = Image.open(an_img_path)
    print("format:", an_img.format)
    print("mode:", an_img.mode)
    print("size:", an_img.size)

The data used for model training is contained in the folder `binary_128x128_2000train`. This folder contains sub-folders `test`, `train`, and `val`.

In [ ]:
# Print the data directory structure
print(os.listdir("../Data"))
print(os.listdir("../Data/binary_128x128_2000train"))

# Set folder variables
proj_dir = "."
dataset_base_dir = '../Data/binary_128x128_2000train/'
TRAIN_DIR = dataset_base_dir + 'train'
VAL_DIR = dataset_base_dir + 'val'
TEST_DIR = dataset_base_dir + 'test'

# Get number of samples in each folder
num_train_samples = len(list_files(os.path.join(proj_dir, TRAIN_DIR)))
num_val_samples = len(list_files(os.path.join(proj_dir, VAL_DIR)))
num_test_samples = len(list_files(os.path.join(proj_dir, TEST_DIR)))

### Check data
The data is split into training, validation, and test sets. The train set is used to train the model, and validation set is used to check performance during training. The test set is held out until the end to check final model performance.

To get an idea of what the data looks like, we load a sample image from the training folder and print its metadata.

In [ ]:
# Print the number of images in train, validation, and test directories
print(num_train_samples, num_val_samples, num_test_samples)

# Load an image and print image metadata
from PIL import Image
sample_img_path = list_files(os.path.join(proj_dir, TRAIN_DIR))[0]

sample_img = Image.open(sample_img_path)
display(sample_img)
print_pil_img_info(sample_img_path)

### Define model

We start with a MobileNet model and modify it for our needs.

In [ ]:
# Set model training variables
batch_size = 32
tile_size = (128,128) 

# Define base model
base_model = applications.MobileNet(weights = "imagenet",
                                 include_top=False,
                                 input_shape = (tile_size[0], tile_size[1], 3))

x = base_model.output
x = layers.Flatten(name = "flatten")(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(2, activation = 'softmax')(x)

model = models.Model(inputs = base_model.input, outputs = predictions)

model.compile(loss = "categorical_crossentropy",
              optimizer = optimizers.Adam(learning_rate = 0.0001),
              metrics = ["accuracy"])

# Uncomment the following line to see the (long) model summary
# print(model.summary())

### Define data generators

Define the data generators used during training and testing. The training data generator includes a few transformations that are applied to images during training.

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect')

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=tile_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode = 'rgb')

validation_generator = test_datagen.flow_from_directory(
        VAL_DIR,
        target_size=tile_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode = 'rgb')

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size = tile_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle = False)

# plot_losses = PlotLosses()

### Train the model

We now train the model.

In [ ]:
NUM_EPOCHS = 6

tic = perf_counter()
model_hist = model.fit(
        train_generator,
        steps_per_epoch= num_train_samples // batch_size,
        epochs=NUM_EPOCHS,
        validation_data=validation_generator,
        validation_steps= num_val_samples // batch_size)
toc = perf_counter()
print(f"Trained model in {toc - tic:0.4f} seconds")

### Plot training history and check model performance

We plot the training and validation set accuracy over the training epochs. We also calculate the accuracy and Matthews correlation coefficient on the test dataset.

In [ ]:
plot_history(model_hist)

y_pred = model.predict(test_generator, steps = None)
y_pred_class = np.array([np.argmax(x) for x in y_pred])
y_truth = test_generator.classes

print("\nModel performance on test dataset:")
print_performance(y_truth, y_pred_class)

### Convert model to ONNX and save

After converting the Keras model to ONNX, we set the `graph.name` property directly.

We also check names of the model input and output. The input name in the model must match the input name used in the `weldingPreProcessing.py` file, otherwise an error will be thrown during the scoring process.

In [ ]:
# Convert from keras to onnx
spec = (tf.TensorSpec((None, tile_size[0], tile_size[1], 3), tf.float32, name="input"),)
model_proto, _ = tf2onnx.convert.from_keras(model = model, input_signature = spec)
# Set the graph name
model_proto.graph.name = 'weldingDefectModel'

# Print the model inputs and outputs
input_names = [n.name for n in model_proto.graph.input]
output_names = [n.name for n in model_proto.graph.output]
print(input_names)
print(output_names)

# Save the model
onnx.save(model_proto, "../Model/weldingDefectModel.onnx")